In [1]:
# step 1: Loading the dataset into python

import os
import pandas as pd

In [2]:
spam_dir = "dataset/spam"
ham_dir = "dataset/ham"

emails = []
labels = []

In [3]:
# loading the emails

for filename in os.listdir(spam_dir):
    with open(os.path.join(spam_dir, filename), 'r', encoding='latin-1') as file:
        emails.append(file.read())
        labels.append(1) # 1 for spam mail

for filename in os.listdir(ham_dir):
    with open(os.path.join(ham_dir , filename), 'r', encoding='latin-1') as file:
        emails.append(file.read())
        labels.append(0) # 0 for ham

# here we just appended the mails and labels to their array

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/spam'

In [ ]:
df = pd.DataFrame({'text': emails, 'label': labels})
print(df.shape)
df.head()

# here we made the df and printed the (Rows, coloumnes) and peaked at the df

(33702, 2)


,text,label
0,Subject: fw : this is the solution i mentioned...,1
1,Subject: adv : space saving computer to replac...,1
2,"Subject: advs\ngreetings ,\ni am benedicta lin...",1
3,Subject: fw : account over due wfxu ppmfztdtet...,1
4,Subject: whats new in summer ? bawled\ncarolyn...,1


In [21]:
# Step 2: proccessing the text

import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize

In [39]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MHOMM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\MHOMM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [28]:
nltk.data.path.clear()
nltk.data.path.append(os.path.join(os.getenv('APPDATA'), 'nltk_data'))


In [40]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_email(text):
    text = BeautifulSoup(text, "html.parser").get_text()      # remove HTML
    text = text.lower()                                       # lowercase
    text = re.sub(r'http\S+|www\S+|[^a-z\s]', '', text)       # remove URLs & punctuation
    words = word_tokenize(text)                          # tokenize
    words = [stemmer.stem(w) for w in words if w not in stop_words]  # stem + remove stopwords
    return " ".join(words)


In [35]:
nltk.data.find('tokenizers/punkt')

FileSystemPathPointer('C:\\Users\\MHOMM\\AppData\\Roaming\\nltk_data\\tokenizers\\punkt')

In [41]:
df['clean_text'] = df['text'].apply(clean_email)
df.head()

C:\Users\MHOMM\AppData\Local\Temp\ipykernel_20140\918289155.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()      # remove HTML


,text,label,clean_text
0,Subject: fw : this is the solution i mentioned...,1,subject fw solut mention lsc oo thank email ad...
1,Subject: adv : space saving computer to replac...,1,subject adv space save comput replac big box d...
2,"Subject: advs\ngreetings ,\ni am benedicta lin...",1,subject adv greet benedicta lindiw hendrick mr...
3,Subject: fw : account over due wfxu ppmfztdtet...,1,subject fw account due wfxu ppmfztdtet elimin ...
4,Subject: whats new in summer ? bawled\ncarolyn...,1,subject what new summer bawl carolyn regret wa...


In [42]:
# Step 3: Feature Extraction (Vectorize)
from sklearn.feature_extraction.text import TfidfVectorizer


In [43]:
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['clean_text']).toarray()
y = df['label']

print("Feature matrix shape:", X.shape)


Feature matrix shape: (33702, 3000)


In [44]:
# Step 4: Trainning the model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [45]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = MultinomialNB()
model.fit(X_train, y_train)

print("✅ Model training completed!")

✅ Model training completed!


In [46]:
# Step 5: evaluating the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [47]:
y_pred = model.predict(X_test)

print("📈 Evaluation Metrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

📈 Evaluation Metrics:
Accuracy: 0.9780
Precision: 0.9716
Recall: 0.9857
F1 Score: 0.9786

Confusion Matrix:
[[3207   99]
 [  49 3386]]


In [1]:
# Saving the model and vectorizer
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("✅ Model and vectorizer saved successfully!")


NameError: name 'model' is not defined